In [1]:
import pandas as pd 
import numpy as np
import warnings
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv',index_col = "srch_destination_id")

In [3]:
train.head(2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
srch_destination_id,,,,,,,,,,,,,,,,,,,,,
8250,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,2,0,1,1,0,3,2,50,628,1
8250,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,2,0,1,1,1,1,2,50,628,1


In [4]:
train["ranking"] = 0

In [5]:
train = train[["hotel_cluster","is_booking","ranking"]]

In [6]:
train.head()

,hotel_cluster,is_booking,ranking
srch_destination_id,,,
8250,1,0,0
8250,1,1,0
8250,1,0,0
14984,80,0,0
14984,21,0,0


In [7]:
train_group = train.groupby(['srch_destination_id','hotel_cluster'])\
['is_booking'].agg(['sum','count'])

In [8]:
train_group

sum  count
srch_destination_id hotel_cluster            
0                   3                0      2
1                   20               4     26
                    30               2     22
                    57               0      1
                    60               0     17
...                                ...    ...
65103               42               0      3
                    50               0      1
65104               5                0      1
                    37               1      2
65107               76               0      1

[396024 rows x 2 columns]

In [9]:
train_group["ranking"] = train_group["sum"] +train_group["count"]

In [10]:
train_group.head()

sum  count  ranking
srch_destination_id hotel_cluster                     
0                   3                0      2        2
1                   20               4     26       30
                    30               2     22       24
                    57               0      1        1
                    60               0     17       17

In [11]:
train_groupp = train_group.drop(['sum','count'], 1)

In [12]:
train_groupp

ranking
srch_destination_id hotel_cluster         
0                   3                    2
1                   20                  30
                    30                  24
                    57                   1
                    60                  17
...                                    ...
65103               42                   3
                    50                   1
65104               5                    1
                    37                   3
65107               76                   1

[396024 rows x 1 columns]

In [13]:
rating_table = pd.pivot_table(train_groupp, values='ranking', index=['srch_destination_id'],
                    columns=['hotel_cluster'])

In [14]:
rating_table = rating_table.fillna(0)

In [15]:
rating_table

hotel_cluster,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
srch_destination_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
65102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def standardized(row):
    new_row = (row - row.mean())/(row.max() - row.min())
    return new_row

In [17]:
ratings_std = rating_table.apply(standardized)

In [18]:

ratings_std.head(3)

hotel_cluster,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
srch_destination_id,,,,,,,,,,,,,,,,,,,,,
0,-0.000179,-0.000025,-0.000346,-0.000256,-0.000386,-0.000315,-0.000892,-0.001043,-0.000182,-0.000428,...,-0.000786,-0.001047,-0.000192,-0.001066,-0.000549,-0.000265,-0.000172,-0.000189,-0.000098,-0.00033
1,-0.000179,-0.000025,-0.000346,-0.000512,-0.000386,-0.000315,-0.000892,-0.001043,-0.000182,-0.000428,...,-0.000786,-0.001047,-0.000192,-0.001066,-0.000549,-0.000265,-0.000172,-0.000189,-0.000098,-0.00033
2,-0.000179,-0.000025,-0.000346,-0.000512,-0.000386,-0.000287,-0.000892,-0.001043,-0.000182,-0.000428,...,-0.000786,-0.001047,-0.000192,-0.001066,-0.000549,-0.000265,-0.000172,-0.000189,-0.000098,-0.00033


In [19]:
item_similarity = cosine_similarity(ratings_std.T)

In [20]:
item_similarity 

array([[ 1.00000000e+00,  2.91523491e-02,  2.53804148e-03, ...,
         7.96839668e-02,  4.52175881e-02,  3.18689674e-02],
       [ 2.91523491e-02,  1.00000000e+00, -2.30249048e-04, ...,
        -1.33301047e-04, -1.02082309e-04,  1.94459717e-02],
       [ 2.53804148e-03, -2.30249048e-04,  1.00000000e+00, ...,
         4.39957613e-01,  1.97934625e-01,  4.26340007e-01],
       ...,
       [ 7.96839668e-02, -1.33301047e-04,  4.39957613e-01, ...,
         1.00000000e+00,  6.86466821e-01,  3.96191582e-01],
       [ 4.52175881e-02, -1.02082309e-04,  1.97934625e-01, ...,
         6.86466821e-01,  1.00000000e+00,  2.29636964e-01],
       [ 3.18689674e-02,  1.94459717e-02,  4.26340007e-01, ...,
         3.96191582e-01,  2.29636964e-01,  1.00000000e+00]])

In [21]:
item_similarity

array([[ 1.00000000e+00,  2.91523491e-02,  2.53804148e-03, ...,
         7.96839668e-02,  4.52175881e-02,  3.18689674e-02],
       [ 2.91523491e-02,  1.00000000e+00, -2.30249048e-04, ...,
        -1.33301047e-04, -1.02082309e-04,  1.94459717e-02],
       [ 2.53804148e-03, -2.30249048e-04,  1.00000000e+00, ...,
         4.39957613e-01,  1.97934625e-01,  4.26340007e-01],
       ...,
       [ 7.96839668e-02, -1.33301047e-04,  4.39957613e-01, ...,
         1.00000000e+00,  6.86466821e-01,  3.96191582e-01],
       [ 4.52175881e-02, -1.02082309e-04,  1.97934625e-01, ...,
         6.86466821e-01,  1.00000000e+00,  2.29636964e-01],
       [ 3.18689674e-02,  1.94459717e-02,  4.26340007e-01, ...,
         3.96191582e-01,  2.29636964e-01,  1.00000000e+00]])

In [22]:
item_similarity_df = pd.DataFrame(item_similarity, index= rating_table.columns,\
                                  columns = rating_table.columns)

In [23]:
item_similarity_df

hotel_cluster,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
hotel_cluster,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.029152,0.002538,0.133047,0.146225,0.176547,0.207643,0.052990,0.026242,0.208303,...,0.280410,0.244750,0.223777,0.296958,0.212759,0.094894,0.753650,0.079684,0.045218,0.031869
1,0.029152,1.000000,-0.000230,0.002292,0.000480,0.000229,-0.000135,0.000186,-0.000146,-0.000222,...,0.282486,0.281789,0.093674,0.009832,0.472449,-0.000191,0.095969,-0.000133,-0.000102,0.019446
2,0.002538,-0.000230,1.000000,0.347328,0.156711,0.571841,0.393826,0.464612,0.251132,0.353169,...,0.385914,0.186184,0.035182,0.411903,0.174622,0.344262,0.011720,0.439958,0.197935,0.426340
3,0.133047,0.002292,0.347328,1.000000,0.384317,0.479745,0.408985,0.302014,0.136801,0.331728,...,0.397219,0.309205,0.164584,0.503806,0.156497,0.294899,0.150199,0.575241,0.596076,0.394081
4,0.146225,0.000480,0.156711,0.384317,1.000000,0.285132,0.358828,0.262049,0.048167,0.346516,...,0.147816,0.499287,0.063534,0.390119,0.410953,0.593840,0.139606,0.460201,0.644950,0.261275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.094894,-0.000191,0.344262,0.294899,0.593840,0.296027,0.444484,0.302666,0.081738,0.486856,...,0.207986,0.644568,0.032618,0.398171,0.527638,1.000000,0.125411,0.332277,0.433157,0.316798
96,0.753650,0.095969,0.011720,0.150199,0.139606,0.166735,0.215480,0.091014,0.031807,0.135888,...,0.282210,0.271597,0.240025,0.303333,0.280652,0.125411,1.000000,0.059842,0.044299,0.055810
97,0.079684,-0.000133,0.439958,0.575241,0.460201,0.568005,0.342394,0.387056,0.371536,0.432720,...,0.229614,0.257301,0.093396,0.378271,0.191079,0.332277,0.059842,1.000000,0.686467,0.396192


In [24]:
def get_similar_hotel_cluster(hotel_cluster, rating):
     similar_hotel = item_similarity_df[hotel_cluster]*(rating -2.5)
     similar_hotel = similar_hotel.sort_values(ascending =False)
     return similar_hotel


In [25]:
# Find simliar hotel cluster based on the user inputed value
print(get_similar_hotel_cluster(30,4))

hotel_cluster
30    1.500000
61    1.147446
58    1.122836
85    1.115802
78    1.082170
        ...   
88    0.004050
24    0.003608
45    0.003587
1     0.003452
74   -0.000696
Name: 30, Length: 100, dtype: float64


In [26]:
user_muti_selection = [(30,4),(45,2),(55,8),(65,2)]


In [27]:
similar_hotel_cluster_df = pd.DataFrame()

In [28]:
 for hotel_cluster, rating in user_muti_selection :
        similar_hotel_cluster_df = similar_hotel_cluster_df.\
        append(get_similar_hotel_cluster(hotel_cluster,rating))
 
     

In [29]:
similar_hotel_cluster_df.head()   

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
30,0.172623,0.003452,0.722243,0.791198,0.126549,0.966243,0.670405,0.588841,0.579885,0.545857,...,0.678703,0.143679,0.290993,0.655272,0.137041,0.203283,0.202059,0.484772,0.153239,1.001201
45,-0.014731,-0.498639,0.000110,-0.001002,0.000102,-0.000127,0.000139,-0.000019,0.000070,0.000106,...,-0.144825,-0.144440,-0.048043,-0.004983,-0.240427,0.000091,-0.049022,0.000064,0.000049,-0.009971
55,0.345156,-0.000736,0.786789,3.470437,3.414302,1.729146,1.227821,0.820711,0.354699,1.480596,...,0.829808,2.021049,0.259724,2.137203,1.395058,2.219941,0.448504,3.380147,4.855711,1.143452
65,-0.130372,0.000031,-0.003959,-0.060685,0.000124,-0.048332,-0.008815,-0.010680,-0.006800,-0.001188,...,-0.027853,-0.006707,-0.102625,-0.034130,-0.006785,0.000006,-0.301375,-0.004920,-0.009054,-0.004784
